In [1]:
# check xgboost version
import xgboost as xgb
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import seaborn as sn
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data_audio_f_mean.csv')

In [3]:
df.shape

(54057, 20)

In [4]:
df_test = df.iloc[:, 5:20]

In [5]:
df_test["target"].replace({"abnormal": "1", "normal": "0"}, inplace=True)
df_test = df_test.astype({"target": int})
y = df_test['target']
df_test = df_test.drop('target', 1)

In [6]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='auto')
# fit and apply the transform
X_over, y_over = oversample.fit_resample(df_test, y)

# Split dataframe into random train and test subsets

X_train, X_test, Y_train, Y_test = train_test_split(
    X_over,
    y_over,
    test_size=0.2,
    random_state=200
)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(70651, 14) (70651,)
(17663, 14) (17663,)


In [28]:
# Split dataframe into random train and test subsets

X_train, X_test, Y_train, Y_test = train_test_split(
    df_test,
    y,
    test_size=0.2,
    random_state=200
)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(43245, 14) (43245,)
(10812, 14) (10812,)


In [7]:
model = xgb.XGBRFClassifier(n_estimators=100, subsample=0.9, colsample_bynode=0.2)

In [8]:
model.fit(X_train, Y_train)

/home/ms/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:49:14] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBRFClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=0.2, colsample_bytree=1, gamma=0, gpu_id=-1,
                importance_type='gain', interaction_constraints='',
                max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=100, n_jobs=12,
                num_parallel_tree=100, objective='binary:logistic',
                random_state=0, reg_alpha=0, scale_pos_weight=1, subsample=0.9,
                tree_method='exact', validate_parameters=1, verbosity=None)

In [9]:
y_pred = model.predict(X_test)

In [10]:
acc_xg = round(model.score(X_test, Y_test) * 100, 2)
acc_xg

69.17

In [11]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.64      0.89      0.74      8861
           1       0.82      0.49      0.61      8802

    accuracy                           0.69     17663
   macro avg       0.73      0.69      0.68     17663
weighted avg       0.73      0.69      0.68     17663



In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,classification_report, confusion_matrix